In [42]:
import csv
import os

def importador_csv(dataset_name, datasets_files_path = "datasets"):
    """procedimiento generico que abre el archivo csv pedido segun el 
    caracter dilimitador y devuelve una lista con su contenido"""
    
    dataset_csv = open(os.path.join(os.path.dirname("__file__"), datasets_files_path, dataset_name), encoding = 'utf8')
    if dataset_name == fifa_dataset_file:
        reader_dataset = csv.reader(dataset_csv, delimiter = ';')
    else:
        reader_dataset = csv.reader(dataset_csv, delimiter = ',')
    header, reader = next(reader_dataset), list(reader_dataset) 
    
    dataset_csv.close()
    return reader

In [43]:
def generar_csv_corregido(nombre_archivo, titulos):
    """procedimiento generico que crea un archivo csv, asigna como primera 
    linea los titulos recibidos como parametro y lo devuelve"""
    
    writer_corregido = open(os.path.join(os.path.dirname("__file__"), datasets_procesados_files_path, nombre_archivo), 'w', encoding='UTF8', newline= '')
    writer_corregido_csv = csv.writer(writer_corregido)
    writer_corregido_csv.writerow(titulos)
    
    return writer_corregido_csv, writer_corregido

In [44]:
def agregar_linea(writer_csv, lista_linea):
    """procedimiento que agrega la linea generada si por lo menos uno de sus elementos no es vacio"""
    
    from collections import Counter
    if lista_linea.count("") != len(lista_linea):
        writer_csv.writerow(lista_linea) 


In [45]:
"""

'files_path' contiene el nombre de la carpeta con todos los archivos del programa
'csv_files_path' contiene el nombre de la ubicación de los archivos csv a utilizar
'spotify_dataset_file' es el dataset del Top 100 de temas musicales de Spotify 2010 a 2019
'lagos_dataset_file' es el dataset de lagos de Argentina
'fifa_dataset_file' es el dataset de jugadores de FIFA 2021

"""
 
datasets_procesados_files_path = "datasets_procesados"
spotify_dataset_file = "Spotify 2010 - 2019 Top 100.csv"
lagos_dataset_file = "Lagos Argentina - Hoja 1.csv"
fifa_dataset_file = "FIFA-21 Complete.csv"

In [46]:
#PROCESAMIENTO DATASET SPOTIFY

def corregir_genero(genero):
    """procedimiento que devuelve la columna de genero corregida: quita espacios en blanco 
    y aplica cambios segun de que palabra se trate"""
    
    import string
    spotify_generos_excepcion = ['edm', 'dfw', 'uk', 'r&b', 'lgbtq+']
    lista_generos = []
    
    """contemplar casos de escritura de generos
    1- si es k-pop
    2- si es genero normal
    3- si es genero en siglas
    4- si es genero en siglas + genero normal
    """
    
    if genero.strip() == 'k-pop':
        lista_generos.append('K-Pop')
    else:
        #si el genero es (2; 3 o 4), divide el string en palabras y pasa a mayus si es en siglas 
        #o pone en mayus la primera si es normal
        lista_palabras = list(genero.strip().split())
        lista_generos = list(map(lambda palabra: palabra.upper() if palabra in spotify_generos_excepcion else palabra.capitalize(), lista_palabras))
   
    return " ".join(lista_generos)

#cargo dataset de spotify
spotify_dataset = importador_csv(spotify_dataset_file)

#genero dataset nuevo con titutlos
lista_titulos_spotify = [
    "Genero", 
    "Tipo de artista", 
    "Año Lanzado", 
    "Año Con Puesto Mas Alto", 
    "BPM", 
    "Artista"
]
spotify_writer_csv, spotify_writer = generar_csv_corregido("spotify_procesado.csv", lista_titulos_spotify)

#cargo el dataset linea por linea corrigiendo y acomodando los elementos pedidos
for linea in spotify_dataset:
    lista_linea = [
        corregir_genero(linea[2]),
        linea[16],
        linea[3],
        linea[15],
        linea[5],
        linea[1]
    ]

    agregar_linea(spotify_writer_csv, lista_linea)
spotify_writer.close()

In [47]:
#PROCESAMIENTO DATASET LAGOS

def conversor_coord_decimal(coord):
    """procedimiento que convierte el contenido de la columna 5 del dataset lagos (coordenadas),
    que esta en formato str, a decimal, y luego lo devuelve a str"""
    
    puntos_cardinales = {'N':1, 'S':-1, 'E': 1, 'O':-1}
    lat = coord.split(" ")[0].replace("°", " ").replace("'", " ").replace('"'," ").split(" ")
    long = coord.split(" ")[1].replace("°", " ").replace("'", " ").replace('"'," ").split(" ")
    lat_pc = lat.pop(-1)
    long_pc = long.pop(-1)
    latitud = (int(lat[0]) + (int(lat[1])/60) + (int(lat[2])/3600)) * puntos_cardinales[lat_pc]
    longitud = (int(long[0]) + (int(long[1])/60) + (int(long[2])/3600)) * puntos_cardinales[long_pc]
    return str(latitud) + " " + str(longitud)

#cargo dataset lagos
lagos_dataset = importador_csv(lagos_dataset_file)


#se carga un dataset nuevo a traves del reader del dataset de lagos
#agregando los elementos en el orden pedido, y realizando las modificaciones 
#en la columna de coordenadas


lista_titulos_lagos = [
    "Ubicación", 
    "Superficie (km²)", 
    "Profundidad máxima (m)", 
    "Profundidad media (m)",
    "Coordenadas", 
    "Nombre"
]
lagos_writer_csv, lagos_writer = generar_csv_corregido("lagos_procesado.csv", lista_titulos_lagos)

#cargo el dataset linea por linea corrigiendo y acomodando los elementos pedidos
for linea in lagos_dataset:
    lista_linea = [
        linea[1].replace(',','.').replace('"', " "),
        linea[2],
        linea[3],
        linea[4],
        conversor_coord_decimal(linea[5]),
        linea[0]
    ]
    agregar_linea(lagos_writer_csv, lista_linea)
lagos_writer.close()

In [48]:
#PROCESAMIENTO DATASET FIFA

def obtener_potencial(potencial):
    """procedimiento que devuelve la calidad del jugador en str segun sus puntos de potencial recibidos"""
    
    if  potencial < 60:
        return "Regular"
    elif potencial >= 60 and potencial <= 79:
        return "Bueno"
    elif potencial > 80 <= 90:
        return "Muy Bueno"
    else:
        return "Sobresaliente"

def posicion_en_espanol(posicion):
    """procedimiento que divide las posiciones del jugador es una lista,
    asigna a cada posicion su nombre correspondiente en español y
    devuelve un string con las posiciones"""
    
    lista_pos = posicion.split("|")
    posiciones = {
        "GK": "Arquero",
        "LB": "Lateral Izquierdo",
        "LCB": "Defensor Central Izquierdo",
        "CB": "Defensor Central",
        "RCB": "Defensor Central Derecho",
        "RB": "Lateral Derecho",
        "LWB": "Carrilero Izquierdo",
        "LDM": "Mediocampista Defensivo Izquierdo",
        "CDM": "Mediocampista Defensivo Central",
        "RDM": "Mediocampista Defensivo Derecho",
        "RWB": "Carrilero Derecho",
        "LM": "Mediocampista Izquierdo",
        "LCM": "Mediocampista Central Izquierdo",
        "CM": "Mediocampista Central",
        "RCM": "Mediocampista Central Derecho",
        "RM": "Mediocampista Derecho",
        "LAM": "Mediocampista Ofensivo Izquierdo",
        "CAM": "Mediocampista Ofensivo Central",
        "RAM": "Mediocampista Ofensivo Derecho",
        "LW": "Extremo Izquierdo",
        "ST": "Segundo Delantero",
        "RW": "Extremo Derecho",
        "LF": "Delantero Izquierdo",
        "CF": "Media Punta",
        "RF": "Delantero Derecho",
    }
    lista_pos_esp = []
    #for pos in lista_pos:
        #lista_pos_esp.append(posiciones[pos])
    lista_pos_esp = list(map(lambda pos: posiciones[pos], lista_pos))
        
    return "|".join(lista_pos_esp)

#cargo dataset de fifa
fifa_dataset = importador_csv(fifa_dataset_file)

#genero dataset nuevo con titulos
lista_titulos_fifa = [
    "Equipo", 
    "Nacionalidad", 
    "Posicion", 
    "Edad", 
    "Potencial", 
    "Nombre"
]
fifa_writer_csv, fifa_writer = generar_csv_corregido("fifa_procesado.csv", lista_titulos_fifa)

#cargo el dataset linea por linea corrigiendo y acomodando los elementos pedidos
for linea in fifa_dataset:
    lista_linea = [
        linea[8].strip(), 
        linea[2], 
        posicion_en_espanol(linea[3]), 
        linea[5], 
        obtener_potencial(int(linea[7])), 
        linea[1]
    ]
    agregar_linea(fifa_writer_csv, lista_linea)
fifa_writer.close()